In [3]:
!pip install torchvision


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:

import numpy as np
import autokeras as ak
from keras.preprocessing.image import ImageDataGenerator
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import v2

In [2]:
import matplotlib.pyplot as plt

In [3]:
data_dir = 'data pics'

# Define the AutoAugment policy
autoaugment_policy = v2.AutoAugmentPolicy.IMAGENET
autoaugment_transform = v2.AutoAugment(autoaugment_policy)

# Use ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale = 1. / 255)

# Define the data generators for training, validation, and testing
train_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/train',
    target_size = (256, 256),
    batch_size = 16,
    class_mode = 'categorical'
)


test_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/test',
    target_size = (256, 256),
    batch_size = 1,
    class_mode = 'categorical',
    shuffle = False
)

Found 322 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


In [7]:


# Initialize empty lists for training and test data and labels
x_train, y_train = [], []
x_test, y_test = [], []
test = []
test1 = []

# Load training data in batches
for _ in range(len(train_generator)):
    batch_x, batch_y = train_generator.next()

    # Apply AutoAugment to each image in the batch
    augmented_images = []
    augmented_images_labels = []
    n = 0
    for image in batch_x:
        # Convert the image to a tensor (required for AutoAugment)
        tensor_image = to_tensor(image)
        
        # plt.imshow(image)
        # plt.title('Original Image')
        # plt.show()
        # Apply AutoAugment policy
        for _ in range(3):  
            augmented_tensor = autoaugment_transform(tensor_image)
            augmented_image = transforms.ToPILImage()(augmented_tensor)
            augmented_images.append(np.array(augmented_image))
            augmented_images_labels.append(np.array(batch_y[n]))
            # plt.imshow(np.array(augmented_image))
            # plt.title('Augmented Image')
            # plt.show()

        n += 1

    x_train.append(batch_x)
    y_train.append(batch_y)
    x_train.append(augmented_images)
    y_train.append(augmented_images_labels)

# Load test data in batches
for _ in range(len(test_generator)):
    batch_x, batch_y = test_generator.next()

    # Apply AutoAugment to each image in the batch
    augmented_images = []
    augmented_images_labels = []
    n = 0
    for image in batch_x:
        # Convert the image to a tensor (required for AutoAugment)
        tensor_image = to_tensor(image)
        
        # plt.imshow(image)
        # plt.title('Original Image')
        # plt.show()
        # Apply AutoAugment policy
        for _ in range(3):
            augmented_tensor = autoaugment_transform(tensor_image)
            augmented_image = transforms.ToPILImage()(augmented_tensor)
            augmented_images.append(np.array(augmented_image))
            augmented_images_labels.append(np.array(batch_y[n]))
            # plt.imshow(np.array(augmented_image))
            # plt.title('Augmented Image')
            # plt.show()

        n += 1

    
    x_test.append(batch_x)
    y_test.append(batch_y)
    x_test.append(augmented_images)
    y_test.append(augmented_images_labels)


In [8]:
# Concatenate the training and test data
x_train = np.concatenate(x_train)
x_test = np.concatenate(x_test)

# Concatenate the labels and then convert to integer labels
y_train = np.concatenate(y_train, axis=0)
y_train = np.argmax(y_train, axis=1)

y_test = np.concatenate(y_test, axis=0)
y_test = np.argmax(y_test, axis=1)


In [9]:
clf = ak.ImageClassifier(
    max_trials = 1,
    objective = 'val_accuracy',
    overwrite = True,
)

# Search for the best model architecture
clf.fit(x_train, y_train, epochs = 10)

Trial 1 Complete [00h 29m 58s]
val_accuracy: 0.8663793206214905

Best val_accuracy So Far: 0.8663793206214905
Total elapsed time: 00h 29m 58s
Epoch 1/10
41/41 [==============================] - 196s 5s/step - loss: 4.2967 - accuracy: 0.6576
Epoch 2/10
41/41 [==============================] - 224s 6s/step - loss: 0.3323 - accuracy: 0.8245
Epoch 3/10
41/41 [==============================] - 201s 5s/step - loss: 0.2989 - accuracy: 0.8300
Epoch 4/10
41/41 [==============================] - 220s 5s/step - loss: 0.2844 - accuracy: 0.8354
Epoch 5/10
41/41 [==============================] - 212s 5s/step - loss: 0.2828 - accuracy: 0.8339
Epoch 6/10
41/41 [==============================] - 197s 5s/step - loss: 0.2787 - accuracy: 0.8346
Epoch 7/10
41/41 [==============================] - 194s 5s/step - loss: 0.2820 - accuracy: 0.8292
Epoch 8/10
41/41 [==============================] - 237s 6s/step - loss: 0.2846 - accuracy: 0.8307
Epoch 9/10
41/41 [==============================] - 223s 5s/step -

INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


In [10]:
results = clf.evaluate(x_test, y_test)
print(results)
print("Test accuracy:", results[1])

7/7 [==============================] - 6s 780ms/step - loss: 0.3001 - accuracy: 0.8287
[0.3001139163970947, 0.8287037014961243]
Test accuracy: 0.8287037014961243


In [11]:
saved_model_path = "AI_model_autoAugment"
clf.export_model().save(saved_model_path)

INFO:tensorflow:Assets written to: AI_model_autoAugment\assets


INFO:tensorflow:Assets written to: AI_model_autoAugment\assets


In [4]:
from keras.models import load_model

In [6]:
saved_model_path = "AI_model_autoAugment"
model = load_model(saved_model_path, custom_objects=ak.CUSTOM_OBJECTS)

In [7]:
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_weights.h5")

In [8]:
import h5py

In [11]:
# Function to recursively visit all datasets in the HDF5 file
def collect_datasets(name, obj):
    if isinstance(obj, h5py.Dataset):
        datasets.append(obj[()])

# Open the HDF5 file in read mode
with h5py.File('model_weights.h5', 'r') as h5_file:
    datasets = []
    # Recursively visit all datasets in the file
    h5_file.visititems(collect_datasets)

# Concatenate all datasets
data = b''.join(datasets)

# Specify the binary file name
bin_file_name = 'model_weights.bin'

# Write the data to the binary file
with open(bin_file_name, 'wb') as bin_file:
    bin_file.write(data)